In [27]:
import pandas as pd
import numpy as np

import os
os.environ['THEANO_FLAGS']='device=cpu'
import pymc3 as pm
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
%%time
root = './input/'
train_df = pd.read_csv(root + 'train.csv')
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"], format='%Y-%m-%d %H:%M:%S')

weather_train_df = pd.read_csv(root + 'weather_train.csv')
test_df = pd.read_csv(root + 'test.csv')
weather_test_df = pd.read_csv(root + 'weather_test.csv')
building_meta_df = pd.read_csv(root + 'building_metadata.csv')
sample_submission = pd.read_csv(root + 'sample_submission.csv')

CPU times: user 20.5 s, sys: 8.85 s, total: 29.4 s
Wall time: 33.6 s


In [29]:
print('Size of train_df data', train_df.shape)
print('Size of weather_train_df data', weather_train_df.shape)
print('Size of weather_test_df data', weather_test_df.shape)
print('Size of building_meta_df data', building_meta_df.shape)

Size of train_df data (20216100, 4)
Size of weather_train_df data (139773, 9)
Size of weather_test_df data (277243, 9)
Size of building_meta_df data (1449, 6)


In [30]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [31]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

weather_train_df = reduce_mem_usage(weather_train_df)
weather_test_df = reduce_mem_usage(weather_test_df)
building_meta_df = reduce_mem_usage(building_meta_df)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [32]:
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


In [33]:
test_df.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00


In [34]:
weather_train_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.000000,6.0,20.00000,NaN,1019.5,0.0,0.000000
1,0,2016-01-01 01:00:00,24.406250,NaN,21.09375,-1.0,1020.0,70.0,1.500000
2,0,2016-01-01 02:00:00,22.796875,2.0,21.09375,0.0,1020.0,0.0,0.000000
3,0,2016-01-01 03:00:00,21.093750,2.0,20.59375,0.0,1020.0,0.0,0.000000
4,0,2016-01-01 04:00:00,20.000000,2.0,20.00000,-1.0,1020.0,250.0,2.599609


In [35]:
weather_test_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
1,0,2017-01-01 01:00:00,17.796875,2.0,12.796875,0.0,1022.0,130.0,3.099609
2,0,2017-01-01 02:00:00,16.093750,0.0,12.796875,0.0,1022.0,140.0,3.099609
3,0,2017-01-01 03:00:00,17.203125,0.0,13.296875,0.0,1022.0,140.0,3.099609
4,0,2017-01-01 04:00:00,16.703125,2.0,13.296875,0.0,1022.5,130.0,2.599609


In [36]:
building_meta_df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


 ### Building DF merge through concat

In [37]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'])
weather_test_df['timestamp'] = pd.to_datetime(weather_test_df['timestamp'])

In [38]:
temp_df = train_df[['building_id']]
temp_df = temp_df.merge(building_meta_df, on=['building_id'], how='left')
del temp_df['building_id']
train_df = pd.concat([train_df, temp_df], axis=1)

temp_df = test_df[['building_id']]
temp_df = temp_df.merge(building_meta_df, on=['building_id'], how='left')

del temp_df['building_id']
test_df = pd.concat([test_df, temp_df], axis=1)
del temp_df, building_meta_df

In [39]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN


In [40]:
test_df.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,0,2017-01-01,0,Education,7432,2008.0,NaN
1,1,1,0,2017-01-01,0,Education,2720,2004.0,NaN
2,2,2,0,2017-01-01,0,Education,5376,1991.0,NaN
3,3,3,0,2017-01-01,0,Education,23685,2002.0,NaN
4,4,4,0,2017-01-01,0,Education,116607,1975.0,NaN


### Weather DF merge over concat

In [41]:
temp_df = train_df[['site_id','timestamp']]
temp_df = temp_df.merge(weather_train_df, on=['site_id','timestamp'], how='left')

del temp_df['site_id'], temp_df['timestamp']
train_df = pd.concat([train_df, temp_df], axis=1)

temp_df = test_df[['site_id','timestamp']]
temp_df = temp_df.merge(weather_test_df, on=['site_id','timestamp'], how='left')

del temp_df['site_id'], temp_df['timestamp']
test_df = pd.concat([test_df, temp_df], axis=1)

del temp_df, weather_train_df, weather_test_df

In [42]:
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0


In [43]:
test_df.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01,0,Education,7432,2008.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
1,1,1,0,2017-01-01,0,Education,2720,2004.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
2,2,2,0,2017-01-01,0,Education,5376,1991.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
3,3,3,0,2017-01-01,0,Education,23685,2002.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
4,4,4,0,2017-01-01,0,Education,116607,1975.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609


In [44]:
print(train_df.shape)
print(test_df.shape)

(20216100, 16)
(41697600, 16)


In [84]:
# find missing values
def find_missing_data(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
    missing_train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_train_data

In [54]:
# drop missing value columns
train_df = train_df.drop(columns=['floor_count'])

In [22]:
# drop missing value by rows
# train_df = train_df.dropna()

In [55]:
# check shape after drop
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,25.0,6.0,20.0,NaN,1019.5,0.0,0.0


In [72]:
site_2 = train_df.loc[(train_df['site_id'] == 2) & (train_df['year_built'] > 2006) & (train_df['year_built'] < 2008)]

In [74]:
site_2.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
166,156,0,2016-01-01 00:00:00,114.709999,2,Public services,43681,2007.0,15.601562,6.0,-5.601562,NaN,1015.5,270.0,3.599609
266,203,0,2016-01-01 00:00:00,241.350006,2,Lodging/residential,63348,2007.0,15.601562,6.0,-5.601562,NaN,1015.5,270.0,3.599609
267,203,1,2016-01-01 00:00:00,181.785995,2,Lodging/residential,63348,2007.0,15.601562,6.0,-5.601562,NaN,1015.5,270.0,3.599609
268,203,3,2016-01-01 00:00:00,99.644096,2,Lodging/residential,63348,2007.0,15.601562,6.0,-5.601562,NaN,1015.5,270.0,3.599609
2467,156,0,2016-01-01 01:00:00,117.870003,2,Public services,43681,2007.0,13.898438,NaN,-5.601562,0.0,1015.5,270.0,4.101562


In [78]:
site_2['building_id'].value_counts()

203    26254
156     8783
Name: building_id, dtype: int64

In [80]:
building_156 = train_df.loc[(train_df['building_id'] == 156) & (train_df['meter'] == 0)]

In [85]:
building_156_missing = find_missing_data(building_156)

In [97]:
building_156 = building_156.drop(columns=['cloud_coverage'])

In [89]:
building_203 = train_df.loc[(train_df['building_id'] == 203) & (train_df['meter'] == 0)]

In [90]:
building_203_missing = find_missing_data(building_203)

In [95]:
building_203 = building_203.drop(columns=['cloud_coverage'])

KeyError: "['cloud_coverage'] not found in axis"

In [98]:
building_156

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,air_temperature,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
166,156,0,2016-01-01 00:00:00,114.709999,2,Public services,43681,2007.0,15.601562,-5.601562,NaN,1015.5,270.0,3.599609
2467,156,0,2016-01-01 01:00:00,117.870003,2,Public services,43681,2007.0,13.898438,-5.601562,0.0,1015.5,270.0,4.101562
4760,156,0,2016-01-01 02:00:00,122.699997,2,Public services,43681,2007.0,13.296875,-5.601562,0.0,1016.0,270.0,3.099609
7059,156,0,2016-01-01 03:00:00,118.419998,2,Public services,43681,2007.0,12.203125,-6.101562,0.0,1016.5,280.0,3.099609
9355,156,0,2016-01-01 04:00:00,130.339996,2,Public services,43681,2007.0,11.703125,-6.699219,0.0,1017.0,270.0,3.099609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20204465,156,0,2016-12-31 19:00:00,107.910004,2,Public services,43681,2007.0,15.000000,11.101562,0.0,1012.5,210.0,3.599609
20206828,156,0,2016-12-31 20:00:00,107.830002,2,Public services,43681,2007.0,16.093750,10.000000,0.0,1011.5,260.0,3.599609
20209194,156,0,2016-12-31 21:00:00,105.510002,2,Public services,43681,2007.0,16.703125,9.398438,0.0,1011.0,240.0,4.601562
20211559,156,0,2016-12-31 22:00:00,105.510002,2,Public services,43681,2007.0,16.703125,10.000000,0.0,1011.0,250.0,3.599609


In [26]:
# y = train_df['meter_reading']
# X = train_df.drop(['meter_reading'], axis=1)

### Fitting a basic linear regression with no dynamic covariates

In [46]:
n_locations = 20
n_timesteps = 100
p_static    = 3
p_dynamic   = 4
error_sd    = 1.5
error_corr  = 0.3